# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api
influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


cuda is available.
cuda is available.


# 1. set param from Front End

In [2]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'step_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':24, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'classification',
        'model_method' : 'LSTM_cf',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6, 'rnn_type':'lstm'}
    }
}

# 2. Backend

In [3]:
# 1. Backend Prameter Setting
from Clust.clust.ML.common import ML_api

# parameter tunning
param = param2
param = ML_api.convert_param_for_backend(param)

In [4]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (mongodb_client, param['ingestion_param_X'])
param['train_data_info'] = train_data_info

In [5]:
# 3. Data Preparation
#train_X_array, train_y_array, val_X_array, val_y_array = ML_api.ML_data_preparation(param, influxdb_client)

from Clust.clust.ML.common import ML_pipeline
# 1. Oirignla data ingestion
data_X, data_y = ML_pipeline.Xy_data_preparation(param['ingestion_param_X'], 
                                             param['data_y_flag'], 
                                             param['ingestion_param_y'],
                                             'ms_all', 
                                             influxdb_client)
# 2. Scaling
dataX_scaled, datay_scaled = ML_pipeline.Xy_data_scaling_train(param['ingestion_param_X']['ms_name'], 
                                                                                 data_X, 
                                                                                 param['ingestion_param_y']['ms_name'], 
                                                                                 data_y, 
                                                                                 param['scaler_param'])



# 3.clean column
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, param['transform_param'])

param['transform_param']

{'bucket_name': 'integration', 'ms_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX', 'feature_list': ['Temperature', 'out_temp', 'sin_hour']}
['Temperature', 'out_temp', 'sin_hour']
Make New scaler File
./scaler/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/minmax/c1b524a5c854f6e561c4764b1dfb4e6f/scaler.pkl
['Temperature']
Make New scaler File


{'data_clean_option': False,
 'split_mode': 'step_split',
 'past_step': 24,
 'future_step': 2,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5}

In [6]:
# 4. split train/Val
split_ratio = 0.8
train_X, val_X, train_y, val_y, param['transform_param']= ML_pipeline.split_data_by_mode(dataX_scaled, 
                                                                                         datay_scaled, 
                                                                                         split_ratio, 
                                                                                         param['transform_param'])

None
None


In [7]:
train_X

,Temperature,out_temp,sin_hour
time,,,
2021-02-01 00:00:00+00:00,0.513455,0.205128,0.500000
2021-02-01 00:10:00+00:00,0.513455,0.206197,0.521568
2021-02-01 00:20:00+00:00,0.513455,0.207265,0.543137
2021-02-01 00:30:00+00:00,0.514182,0.208333,0.564705
2021-02-01 00:40:00+00:00,0.528000,0.209402,0.586273
...,...,...,...
2021-03-02 12:50:00+00:00,0.736000,0.592415,0.392159
2021-03-02 13:00:00+00:00,0.731636,0.576923,0.370590
2021-03-02 13:10:00+00:00,0.728000,0.559295,0.350492


In [8]:
print(len(val_X)/128)
print(len(val_y))

8.3203125
1065


In [9]:
# 5. Transform array style
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                        train_X, 
                                                                        train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                    val_X, 
                                                                    val_y)

nan_limit_num:  12
Original num: 4258 Final num: 4164 NaN num: 94
nan_limit_num:  12
Original num: 1065 Final num: 1039 NaN num: 26


In [10]:
# 4.Training
param = ML_api.ML_training(train_X_array,  train_y_array, val_X_array, val_y_array, param)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/LSTM_rg/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/model.pkl
Start training model


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu

In [ ]:
param

In [ ]:
# save meta
from Clust.clust.ML.tool import meta
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param)
meta.save_model_meta_into_local(json_file_path, model_meta)

In [ ]:
param

In [ ]:
param['model_info']